In [1]:
!pip install pyqubo

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [440]:
#頂点数
N_VER = 10
#色の数
N_COLOR = 5
#辺の数
N_EDGE = 20

In [445]:
#グラフを作成します
graph = []
for i in range(N_EDGE):
    a = random.randint(0, N_VER-1)
    b = random.randint(0, N_VER-1)
    if a!=b and (a,b) not in graph and (b,a) not in graph:
        graph.append((a,b))
print(graph)

[(5, 9), (5, 6), (1, 5), (4, 0), (0, 7), (7, 5), (6, 1), (6, 3), (0, 1), (9, 3), (1, 7), (9, 6), (0, 3), (7, 4), (8, 9), (0, 8), (4, 3)]


In [446]:
#PYQUBOをimportします
from pyqubo import Array, Constraint, solve_qubo

In [447]:
x = Array.create('x', shape=(N_VER,N_COLOR), vartype='BINARY')

In [448]:
# ハミルトニアンの第一項 
H_A = Constraint(sum((1-sum(x[v,i] for i in range(1,N_COLOR)))**2 for v in range(N_VER)), label='HA')
# ハミルトニアンの第二項 
H_B = sum(sum(x[u,i] * x[v,i] for i in range (1,N_COLOR)) for u,v in graph)
# ハミルトニアン全体を定義します
Q = H_A+H_B

In [449]:
model = Q.compile()
qubo, offset = model.to_qubo()

In [450]:
# PyQUBOに備わっているSAを使います
raw_solution = solve_qubo(qubo)
print(raw_solution)

{'x[0][1]': 1, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 1, 'x[2][1]': 0, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 1, 'x[3][1]': 0, 'x[3][2]': 1, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 1, 'x[5][1]': 0, 'x[5][2]': 0, 'x[5][3]': 1, 'x[5][4]': 0, 'x[6][1]': 1, 'x[6][2]': 0, 'x[6][3]': 0, 'x[6][4]': 0, 'x[7][1]': 0, 'x[7][2]': 1, 'x[7][3]': 0, 'x[7][4]': 0, 'x[8][1]': 0, 'x[8][2]': 0, 'x[8][3]': 1, 'x[8][4]': 0, 'x[9][1]': 0, 'x[9][2]': 0, 'x[9][3]': 0, 'x[9][4]': 1}


In [456]:
#結果の表示
decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY")
print(decoded_solution['x'])

{0: {1: 1, 2: 0, 3: 0, 4: 0}, 1: {1: 0, 2: 0, 3: 0, 4: 1}, 2: {1: 0, 2: 0, 3: 0, 4: 1}, 3: {1: 0, 2: 1, 3: 0, 4: 0}, 4: {1: 0, 2: 0, 3: 0, 4: 1}, 5: {1: 0, 2: 0, 3: 1, 4: 0}, 6: {1: 1, 2: 0, 3: 0, 4: 0}, 7: {1: 0, 2: 1, 3: 0, 4: 0}, 8: {1: 0, 2: 0, 3: 1, 4: 0}, 9: {1: 0, 2: 0, 3: 0, 4: 1}}


In [452]:
#制約を守れているか見ます
print(broken)

{}


In [453]:
# OpenJijのインポートをします
import openjij as oj
# SQAを使います。
sampler = oj.SQASampler()
# PYQUBOで使ったquboを使います。
response = sampler.sample_qubo(Q=qubo)

In [455]:
# エネルギーが一番低い状態を取り出します。
dict_solution = response.first.sample
# デコードします。
decoded_solution, broken, energy = model.decode_solution(dict_solution, vartype="BINARY")
# 辞書型をsortして見やすくする処理を追加します。
x_solution = dict(sorted(decoded_solution['x'].items()))
print({key:dict(sorted(value.items())) for key, value in x_solution.items()})

{0: {1: 0, 2: 0, 3: 1, 4: 0}, 1: {1: 0, 2: 0, 3: 0, 4: 1}, 2: {1: 0, 2: 0, 3: 1, 4: 0}, 3: {1: 0, 2: 0, 3: 0, 4: 1}, 4: {1: 0, 2: 1, 3: 0, 4: 0}, 5: {1: 0, 2: 1, 3: 0, 4: 0}, 6: {1: 0, 2: 0, 3: 1, 4: 0}, 7: {1: 1, 2: 0, 3: 0, 4: 0}, 8: {1: 0, 2: 1, 3: 0, 4: 0}, 9: {1: 1, 2: 0, 3: 0, 4: 0}}
